# Helper function

In [27]:
def Get_table(url, paylpad, headers):
    dic = json.loads(payload, strict=False)
    res = requests.post(url = url, json = dic, headers = headers)
    dic = res.json()
    entries = dic['data']['results']['listings']
    df = pd.json_normalize(entries)
    return(df)



def clean_nested(df):
    amenities = []
    for i in range(0,len(df)):
        item_list = df['amenitiesBadges'][i]
        temp_list = [item['name'] for item in item_list]
        amenities.append(temp_list)
    df['amenities'] = pd.Series(amenities)

    image_url = []
    for i in range(0,len(df)):
        item_list = df['images'][i]
        temp_list = [item['c6_uri'] for item in item_list]
        image_url.append(temp_list)
    df['image_url'] = pd.Series(image_url)

    rankedBadges_parsed = []
    for i in range(0,len(df)):
        item_list = df['rankedBadges'][i]
        temp_list = [item['name'] for item in item_list]
        rankedBadges_parsed.append(temp_list)

    df['rankedBadges_parsed'] = pd.Series(rankedBadges_parsed)
    return(df)

def select_var(df):
    variables = ['detailPageUrl', 'instantBookable','propertyId' , 'webRatingBadges', 'bedrooms', 'propertyType', 'sleeps', 'petsAllowed', 'averageRating', 'reviewCount', 'propertyMetadata.headline', 'unitMetadata.unitName', 'bathrooms.full', 'bathrooms.half', 'bathrooms.toiletOnly', 'spaces.spacesSummary.area.areaValue','spaces.spacesSummary.bedCountDisplay','geoDistance.text', 'geoDistance.relationType', 'priceSummary.priceAccurate', 'priceSummary.edapEventJson', 'prices.perNight.amount', 'geoCode.latitude', 'geoCode.longitude', 'amenities', 'image_url']
    return(df[variables])

In [10]:
from IPython.core.display import json
import json
import requests
import pandas as pd
import numpy as np
from time import sleep

In [11]:

# iterate through pages by changing 'page'
url =  'https://www.vrbo.com/serp/g'
headers = {'Content-Type': 'application/json'}
df = pd.DataFrame()
for i in range(1,30):
    payload = r"""
{"operationName":"SearchRequestQuery","variables":{"filterCounts":true,"request":{"paging":{"page":%s,"pageSize":50},"filterVersion":"1","filters":[],"coreFilters":{"maxBathrooms":null,"maxBedrooms":null,"maxNightlyPrice":null,"maxTotalPrice":null,"minBathrooms":0,"minBedrooms":0,"minNightlyPrice":0,"minTotalPrice":null,"pets":0},"q":"miami-florida-united-states-of-america"},"optimizedBreadcrumb":false,"vrbo_web_global_messaging_banner":true},"extensions":{"isPageLoadSearch":true},"query":"query SearchRequestQuery($request: SearchResultRequest!, $filterCounts: Boolean!, $optimizedBreadcrumb: Boolean!, $vrbo_web_global_messaging_banner: Boolean!) {\n  results: search(request: $request) {\n    ...querySelectionSet\n    ...DestinationBreadcrumbsSearchResult\n    ...DestinationMessageSearchResult\n    ...FilterCountsSearchRequestResult\n    ...HitCollectionSearchResult\n    ...ADLSearchResult\n    ...MapSearchResult\n    ...ExpandedGroupsSearchResult\n    ...PagerSearchResult\n    ...SearchTermCarouselSearchResult\n    ...InternalToolsSearchResult\n    ...SEOMetaDataParamsSearchResult\n    ...GlobalInlineMessageSearchResult\n    ...GlobalBannerContainerSearchResult @include(if: $vrbo_web_global_messaging_banner)\n    __typename\n  }\n}\n\nfragment querySelectionSet on SearchResult {\n  id\n  typeaheadSuggestion {\n    uuid\n    term\n    name\n    __typename\n  }\n  geography {\n    lbsId\n    gaiaId\n    location {\n      latitude\n      longitude\n      __typename\n    }\n    isGeocoded\n    shouldShowMapCentralPin\n    __typename\n  }\n  propertyRedirectUrl\n  __typename\n}\n\nfragment DestinationBreadcrumbsSearchResult on SearchResult {\n  destination(optimizedBreadcrumb: $optimizedBreadcrumb) {\n    breadcrumbs {\n      name\n      url\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment HitCollectionSearchResult on SearchResult {\n  page\n  pageSize\n  pageCount\n  queryUUID\n  percentBooked {\n    currentPercentBooked\n    __typename\n  }\n  listings {\n    ...HitListing\n    __typename\n  }\n  resultCount\n  pinnedListing {\n    headline\n    listing {\n      ...HitListing\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment HitListing on Listing {\n  virtualTourBadge {\n    name\n    id\n    helpText\n    __typename\n  }\n  amenitiesBadges {\n    name\n    id\n    helpText\n    __typename\n  }\n  images {\n    altText\n    c6_uri\n    c9_uri\n    mab {\n      banditId\n      payloadId\n      campaignId\n      cached\n      arm {\n        level\n        imageUrl\n        categoryName\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  ...HitInfoListing\n  __typename\n}\n\nfragment HitInfoListing on Listing {\n  listingId\n  ...HitInfoDesktopListing\n  ...HitInfoMobileListing\n  ...PriceListing\n  __typename\n}\n\nfragment HitInfoDesktopListing on Listing {\n  detailPageUrl\n  instantBookable\n  minStayRange {\n    minStayHigh\n    minStayLow\n    __typename\n  }\n  listingId\n  listingNumber\n  rankedBadges(rankingStrategy: SERP) {\n    id\n    helpText\n    name\n    __typename\n  }\n  propertyId\n  propertyMetadata {\n    headline\n    __typename\n  }\n  superlativesBadges: rankedBadges(rankingStrategy: SERP_SUPERLATIVES) {\n    id\n    helpText\n    name\n    __typename\n  }\n  unitMetadata {\n    unitName\n    __typename\n  }\n  webRatingBadges: rankedBadges(rankingStrategy: SRP_WEB_RATING) {\n    id\n    helpText\n    name\n    __typename\n  }\n  ...DetailsListing\n  ...GeoDistanceListing\n  ...PriceListing\n  ...RatingListing\n  __typename\n}\n\nfragment DetailsListing on Listing {\n  bathrooms {\n    full\n    half\n    toiletOnly\n    __typename\n  }\n  bedrooms\n  propertyType\n  sleeps\n  petsAllowed\n  spaces {\n    spacesSummary {\n      area {\n        areaValue\n        __typename\n      }\n      bedCountDisplay\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment GeoDistanceListing on Listing {\n  geoDistance {\n    text\n    relationType\n    __typename\n  }\n  __typename\n}\n\nfragment PriceListing on Listing {\n  priceSummary: priceSummary {\n    priceAccurate\n    ...PriceSummaryTravelerPriceSummary\n    __typename\n  }\n  priceSummarySecondary: priceSummary(summary: \"displayPriceSecondary\") {\n    ...PriceSummaryTravelerPriceSummary\n    __typename\n  }\n  priceLabel: priceSummary(summary: \"priceLabel\") {\n    priceTypeId\n    pricePeriodDescription\n    __typename\n  }\n  prices {\n    ...VrboTravelerPriceSummary\n    __typename\n  }\n  __typename\n}\n\nfragment PriceSummaryTravelerPriceSummary on TravelerPriceSummary {\n  priceTypeId\n  edapEventJson\n  formattedAmount\n  roundedFormattedAmount\n  pricePeriodDescription\n  __typename\n}\n\nfragment VrboTravelerPriceSummary on PriceSummary {\n  perNight {\n    amount\n    formattedAmount\n    roundedFormattedAmount\n    pricePeriodDescription\n    __typename\n  }\n  total {\n    amount\n    formattedAmount\n    roundedFormattedAmount\n    pricePeriodDescription\n    __typename\n  }\n  label\n  mainPrice\n  __typename\n}\n\nfragment RatingListing on Listing {\n  averageRating\n  reviewCount\n  __typename\n}\n\nfragment HitInfoMobileListing on Listing {\n  detailPageUrl\n  instantBookable\n  minStayRange {\n    minStayHigh\n    minStayLow\n    __typename\n  }\n  listingId\n  listingNumber\n  rankedBadges(rankingStrategy: SERP) {\n    id\n    helpText\n    name\n    __typename\n  }\n  propertyId\n  propertyMetadata {\n    headline\n    __typename\n  }\n  superlativesBadges: rankedBadges(rankingStrategy: SERP_SUPERLATIVES) {\n    id\n    helpText\n    name\n    __typename\n  }\n  unitMetadata {\n    unitName\n    __typename\n  }\n  webRatingBadges: rankedBadges(rankingStrategy: SRP_WEB_RATING) {\n    id\n    helpText\n    name\n    __typename\n  }\n  ...DetailsListing\n  ...GeoDistanceListing\n  ...PriceListing\n  ...RatingListing\n  __typename\n}\n\nfragment ExpandedGroupsSearchResult on SearchResult {\n  expandedGroups {\n    ...ExpandedGroupExpandedGroup\n    __typename\n  }\n  __typename\n}\n\nfragment ExpandedGroupExpandedGroup on ExpandedGroup {\n  listings {\n    ...HitListing\n    ...MapHitListing\n    __typename\n  }\n  mapViewport {\n    neLat\n    neLong\n    swLat\n    swLong\n    __typename\n  }\n  __typename\n}\n\nfragment MapHitListing on Listing {\n  ...HitListing\n  geoCode {\n    latitude\n    longitude\n    __typename\n  }\n  __typename\n}\n\nfragment FilterCountsSearchRequestResult on SearchResult {\n  id\n  resultCount\n  filterGroups {\n    groupInfo {\n      name\n      id\n      __typename\n    }\n    filters {\n      count @include(if: $filterCounts)\n      checked\n      filter {\n        id\n        name\n        refineByQueryArgument\n        description\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment MapSearchResult on SearchResult {\n  mapViewport {\n    neLat\n    neLong\n    swLat\n    swLong\n    __typename\n  }\n  page\n  pageSize\n  listings {\n    ...MapHitListing\n    __typename\n  }\n  pinnedListing {\n    listing {\n      ...MapHitListing\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment PagerSearchResult on SearchResult {\n  fromRecord\n  toRecord\n  pageSize\n  pageCount\n  page\n  resultCount\n  __typename\n}\n\nfragment DestinationMessageSearchResult on SearchResult {\n  destinationMessage(assetVersion: 4) {\n    iconTitleText {\n      title\n      message\n      icon\n      messageValueType\n      link {\n        linkText\n        linkHref\n        __typename\n      }\n      __typename\n    }\n    ...DestinationMessageDestinationMessage\n    __typename\n  }\n  __typename\n}\n\nfragment DestinationMessageDestinationMessage on DestinationMessage {\n  iconText {\n    message\n    icon\n    messageValueType\n    __typename\n  }\n  __typename\n}\n\nfragment ADLSearchResult on SearchResult {\n  parsedParams {\n    q\n    coreFilters {\n      adults\n      children\n      pets\n      minBedrooms\n      maxBedrooms\n      minBathrooms\n      maxBathrooms\n      minNightlyPrice\n      maxNightlyPrice\n      minSleeps\n      __typename\n    }\n    dates {\n      arrivalDate\n      departureDate\n      __typename\n    }\n    sort\n    __typename\n  }\n  page\n  pageSize\n  pageCount\n  resultCount\n  fromRecord\n  toRecord\n  pinnedListing {\n    listing {\n      listingId\n      __typename\n    }\n    __typename\n  }\n  listings {\n    listingId\n    __typename\n  }\n  filterGroups {\n    filters {\n      checked\n      filter {\n        groupId\n        id\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  geography {\n    lbsId\n    name\n    description\n    location {\n      latitude\n      longitude\n      __typename\n    }\n    primaryGeoType\n    breadcrumbs {\n      name\n      countryCode\n      location {\n        latitude\n        longitude\n        __typename\n      }\n      primaryGeoType\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment SearchTermCarouselSearchResult on SearchResult {\n  discoveryXploreFeeds {\n    results {\n      id\n      title\n      items {\n        ... on SearchDiscoveryFeedItem {\n          type\n          imageHref\n          place {\n            uuid\n            name {\n              full\n              simple\n              __typename\n            }\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  typeaheadSuggestion {\n    name\n    __typename\n  }\n  __typename\n}\n\nfragment InternalToolsSearchResult on SearchResult {\n  internalTools {\n    searchServiceUrl\n    __typename\n  }\n  __typename\n}\n\nfragment SEOMetaDataParamsSearchResult on SearchResult {\n  page\n  resultCount\n  pageSize\n  geography {\n    name\n    lbsId\n    breadcrumbs {\n      name\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment GlobalInlineMessageSearchResult on SearchResult {\n  globalMessages {\n    ...GlobalInlineAlertGlobalMessages\n    __typename\n  }\n  __typename\n}\n\nfragment GlobalInlineAlertGlobalMessages on GlobalMessages {\n  alert {\n    action {\n      link {\n        href\n        text {\n          value\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    body {\n      text {\n        value\n        __typename\n      }\n      link {\n        href\n        text {\n          value\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    id\n    severity\n    title {\n      value\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment GlobalBannerContainerSearchResult on SearchResult {\n  globalMessages {\n    ...GlobalBannerGlobalMessages\n    __typename\n  }\n  __typename\n}\n\nfragment GlobalBannerGlobalMessages on GlobalMessages {\n  banner {\n    body {\n      text {\n        value\n        __typename\n      }\n      link {\n        href\n        text {\n          value\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    id\n    severity\n    title {\n      value\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n"}"""%i
    df_temp = Get_table(url=url, paylpad=payload, headers=headers)
    sleep(1)
    df = df.append(df_temp, ignore_index = True)



TypeError: 'NoneType' object is not subscriptable

In [28]:
df = clean_nested(df=df)
df = select_var(df)

KeyError: 'amenitiesBadges'

In [25]:
df.to_csv('/Users/yoshi/Dropbox/Mac/Desktop/Cohome_Intern/Data/reference.csv')

In [29]:
df

,detailPageUrl,instantBookable,propertyId,webRatingBadges,bedrooms,propertyType,sleeps,petsAllowed,averageRating,reviewCount,...,spaces.spacesSummary.bedCountDisplay,geoDistance.text,geoDistance.relationType,priceSummary.priceAccurate,priceSummary.edapEventJson,prices.perNight.amount,geoCode.latitude,geoCode.longitude,amenities,image_url
0,/2990193?unitId=3562233&noDates=true,True,2990193,[],1,Apartment,2,True,0.000000,0,...,1 bed,2.3 mi to Miami center,PROVIDED_PLACE_LOCATION_CROW_FLIES,None,"{""unituuid"":""/units/0004/1dd07cb3-0ca7-4b22-9a...",95.0,25.807457,-80.194506,"[Pool, Pets Welcome, Air Conditioning, Interne...",[https://media.vrbo.com/lodging/85000000/84170...
1,/9724266ha?unitId=8806554&noDates=true,True,9724266ha,[],1,Condo,4,False,0.000000,0,...,2 beds,8 min drive to the beach,GAIA_NEARBY_PLACES,None,"{""unituuid"":""/units/0000/3261b6a3-0615-44d7-80...",210.0,25.760372,-80.190197,"[Hot Tub, Pool, Air Conditioning, No Smoking, ...",[https://media.vrbo.com/lodging/85000000/84060...
2,/2104375?unitId=2668908&noDates=true,True,2104375,[],1,Apartment,4,False,4.750000,4,...,3 beds,8 min drive to the beach,GAIA_NEARBY_PLACES,None,"{""unituuid"":""/units/0004/9039f387-b33a-46cf-93...",175.0,25.765574,-80.194303,"[Hot Tub, Pool, Air Conditioning, No Smoking, ...",[https://media.vrbo.com/lodging/60000000/59800...
3,/2069358?unitId=2633890&noDates=true,False,2069358,[],2,Townhome,6,False,4.500000,102,...,3 beds,2.6 mi to Miami center,PROVIDED_PLACE_LOCATION_CROW_FLIES,None,"{""unituuid"":""/units/0004/5080b1b4-d111-4492-b0...",133.0,25.807072,-80.204062,"[Air Conditioning, No Smoking, Internet, TV, W...",[https://media.vrbo.com/lodging/59000000/58060...
4,/2069280?unitId=2633812&noDates=true,False,2069280,[],3,Townhome,10,False,4.650000,100,...,5 beds,2.6 mi to Miami center,PROVIDED_PLACE_LOCATION_CROW_FLIES,None,"{""unituuid"":""/units/0004/94fc1fac-4431-41fa-98...",212.0,25.806472,-80.199106,"[Air Conditioning, No Smoking, Internet, TV, W...",[https://media.vrbo.com/lodging/59000000/58060...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,/2844692?unitId=3416725&noDates=true,True,2844692,[],1,Apartment,4,False,0.000000,0,...,2 beds,8 min drive to the beach,GAIA_NEARBY_PLACES,None,"{""unituuid"":""/units/0004/6dec3013-89c7-456d-b7...",184.0,25.774271,-80.196217,"[Pool, Air Conditioning, No Smoking, Internet,...",[https://media.vrbo.com/lodging/80000000/79080...
496,/3003786?unitId=3575830&noDates=true,False,3003786,[],2,Condo,6,True,0.000000,0,...,3 beds,0.2 mi to Miami center,PROVIDED_PLACE_LOCATION_CROW_FLIES,None,"{""unituuid"":""/units/0004/47dd07a5-e73b-4d3a-92...",150.0,25.766972,-80.188998,"[Pool, Pets Welcome, Air Conditioning, No Smok...",[https://media.vrbo.com/lodging/86000000/85530...
497,/11043930ha?unitId=6075189&noDates=true,True,11043930ha,[],3,House,8,True,4.200000,5,...,3 beds,4.1 mi to Miami center,PROVIDED_PLACE_LOCATION_CROW_FLIES,None,"{""unituuid"":""/units/0001/140a7874-4d87-4616-89...",275.0,25.761997,-80.256790,"[Pets Welcome, Air Conditioning, No Smoking, I...",[https://media.vrbo.com/lodging/78000000/77120...
498,/2303456?unitId=2869588&noDates=true,True,2303456,[],2,Condo,6,False,4.307692,26,...,None,10 min drive to the beach,GAIA_NEARBY_PLACES,None,"{""unituuid"":""/units/0004/d66d06ff-349c-4a27-b8...",213.0,25.783912,-80.213875,"[Pool, Air Conditioning, No Smoking, Internet,...",[https://media.vrbo.com/lodging/68000000/67420...
